In [20]:
import pandas as pd
from heapq import *

In [21]:
direct_df = pd.read_csv('https://github.com/tnpb-1st/if684-projeto-busca-estrela/raw/master/real.csv', index_col=0, decimal=',').astype(float)
real_df = pd.read_csv('https://github.com/tnpb-1st/if684-projeto-busca-estrela/raw/master/real.csv', index_col=0, decimal=',').astype(float)
lines_df = pd.read_csv('https://github.com/tnpb-1st/if684-projeto-busca-estrela/raw/master/colors.csv', index_col=0)

In [22]:
direct_df

,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,E11,E12,E13,E14
E1,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E2,20.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,20.0,7.0,NaN,NaN,NaN,NaN
E3,NaN,17.0,NaN,12.6,NaN,NaN,NaN,NaN,18.8,NaN,NaN,NaN,37.4,NaN
E4,NaN,NaN,12.6,NaN,26.0,NaN,NaN,30.6,NaN,NaN,NaN,NaN,25.6,NaN
E5,NaN,NaN,NaN,26.0,NaN,6.0,4.8,60.0,NaN,NaN,NaN,NaN,NaN,NaN
E6,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E7,NaN,NaN,NaN,NaN,4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E8,NaN,NaN,NaN,30.6,60.0,NaN,NaN,NaN,19.2,NaN,NaN,12.8,NaN,NaN
E9,NaN,20.0,18.8,NaN,NaN,NaN,NaN,19.2,NaN,NaN,24.4,NaN,NaN,NaN
E10,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
class Mapa:
  def __init__(self, tempo_h, tempo_real, linhas):
    self.tempo_h = tempo_h
    self.tempo_real = tempo_real
    self.linhas = linhas
      
  def get_h_time(self, estacao_a, estacao_b):
    return self.tempo_h.at[estacao_a, estacao_b]
  
  def get_real_time(self, estacao_a, estacao_b):
    return self.tempo_real.at[estacao_a, estacao_b]
  
  def pegar_linhas(self, estacao_a, estacao_b):
    return self.linhas.at[estacao_a, estacao_b]

  def pegar_fronteira(self, estacao):
    return self.tempo_real[self.tempo_real[estacao].notna()].index.to_list()

In [24]:
def calcula_caminho(mapa, s0, sf): 
  iterador = 1
  est_visitadas = []
  queue = [(0, 0, None, s0)]
  last_est = {s0: (None, None)} # ultima estação
  while len(queue) > 0:
    print(f'Fronteira na iteração {iterador}: {[queue]}')
    
    f, g, linha, estacao = heappop(queue)
    est_visitadas.append(estacao)
    
    if estacao == sf:
      break
    for borda_estacao in mapa.pegar_fronteira(estacao):
      if not borda_estacao in est_visitadas:
        linha_borda = mapa.pegar_linhas(estacao, borda_estacao)
        
        if linha == None or linha == linha_borda:
          borda_g = g + mapa.get_real_time(estacao, borda_estacao)
        else:
          borda_g = g + mapa.get_real_time(estacao, borda_estacao) + 4.0
        
        borda_f = borda_g + mapa.get_h_time(sf, borda_estacao)

        if borda_estacao in last_est.keys():
          if last_est[borda_estacao][1] > borda_g:
            last_est[borda_estacao] = (estacao, borda_g)
        else:
          last_est[borda_estacao] = (estacao, borda_g) 
        heappush(queue, (borda_f, borda_g, linha_borda, borda_estacao))
    
    iterador += 1
    
  caminho = []
  while estacao != None: 
    caminho.insert(0, estacao)
    estacao = last_est[estacao][0]
  
  s_out = '\\' + '=' * 80 + '|'
  print(s_out)
  print('Caminho:', ' -> '.join(caminho))
  print(f'Duração: {g:.2f} minutos')

In [25]:
metro_de_paris = Mapa(direct_df, real_df, lines_df)

In [26]:
calcula_caminho(metro_de_paris, 'E1', 'E11')

Fronteira na iteração 1: [[(0, 0, None, 'E1')]]
Fronteira na iteração 2: [[(nan, 20.0, 'blue', 'E2')]]
Fronteira na iteração 3: [[(nan, 37.0, 'blue', 'E3'), (68.4, 44.0, 'yellow', 'E9'), (nan, 31.0, 'yellow', 'E10')]]
Fronteira na iteração 4: [[(68.4, 44.0, 'yellow', 'E9'), (nan, 31.0, 'yellow', 'E10'), (nan, 49.6, 'blue', 'E4'), (84.19999999999999, 59.8, 'red', 'E9'), (nan, 78.4, 'red', 'E13')]]
Fronteira na iteração 5: [[(nan, 49.6, 'blue', 'E4'), (nan, 31.0, 'yellow', 'E10'), (nan, 78.4, 'red', 'E13'), (84.19999999999999, 59.8, 'red', 'E9'), (nan, 63.2, 'yellow', 'E8'), (nan, 72.4, 'red', 'E11')]]
Fronteira na iteração 6: [[(nan, 78.4, 'red', 'E13'), (nan, 31.0, 'yellow', 'E10'), (nan, 72.4, 'red', 'E11'), (84.19999999999999, 59.8, 'red', 'E9'), (nan, 63.2, 'yellow', 'E8'), (nan, 75.6, 'blue', 'E5'), (nan, 84.2, 'green', 'E8'), (nan, 79.2, 'green', 'E13')]]
Fronteira na iteração 7: [[(nan, 72.4, 'red', 'E11'), (nan, 31.0, 'yellow', 'E10'), (nan, 84.2, 'green', 'E8'), (84.19999999999